In [20]:
import pandas as pd
import numpy as np

from utils import (
    execute_query,
    create_table_from_df,
    read_sql_file,
    read_json_file
)

# Leer credenciales

In [21]:
# credenciales = {
#     'host' : "localhost",
#     'dbname' : "dvdrental",
#     'user' : "postgres",
#     'password' : "",
#     'port' : "5432"
# }

credenciales = read_json_file("credentials.json")

# Queries

In [25]:
query = """
SELECT 
*
FROM film;
"""
df_resultado = execute_query(query, credenciales)

In [26]:
sql_query = read_sql_file("queries/ejemplo1.sql")
df_resultado_2 = execute_query(sql_query, credenciales)

# Crear tablas

## Generar datos de prueba

In [28]:
# Generar 100 filas de datos
np.random.seed(42)  # Fijar semilla para reproducibilidad

num_rows = 100

data = {
    "sale_id": np.arange(1, num_rows + 1),  # IDs de 1 a 100
    "customer_id": np.random.randint(100, 110, size=num_rows),  # 10 clientes (100-109)
    "category": np.random.choice(["Electronics", "Clothes", "Books", "Home", "Beauty", "Food"], size=num_rows),
    "amount": np.random.randint(10, 1000, size=num_rows),  # Montos entre 10 y 1000
    "sale_date": pd.date_range(start="2024-01-01", periods=num_rows, freq="D")  # Fechas consecutivas
}

test_data = pd.DataFrame(data)

# Mostrar primeras filas
test_data.head()

,sale_id,customer_id,category,amount,sale_date
0,1,106,Home,229,2024-01-01
1,2,103,Clothes,575,2024-01-02
2,3,107,Electronics,864,2024-01-03
3,4,104,Food,745,2024-01-04
4,5,106,Beauty,234,2024-01-05


In [18]:
create_table_from_df(test_data,'datos_transaccionales',credenciales)

Tabla 'datos_transaccionales' creada exitosamente en PostgreSQL.


In [29]:
execute_query('SELECT * FROM datos_transaccionales limit 1;', credenciales)

,sale_id,customer_id,category,amount,sale_date
0,1,106,Home,229,2024-01-01 00:00:00


In [40]:
(
    test_data
    .sort_values('sale_date')
    [['sale_date','amount']]
    .head()
)

,sale_date,amount
0,2024-01-01,229
1,2024-01-02,575
2,2024-01-03,864
3,2024-01-04,745
4,2024-01-05,234


In [41]:
(
    test_data
    .sort_values('sale_date')
    ['amount']
    .rolling(2,1)
    .mean()
    .head()
)

0    229.0
1    402.0
2    719.5
3    804.5
4    489.5
Name: amount, dtype: float64

In [34]:
test_data.groupby(['category'])['amount'].mean().shape

(6,)

# Ejercicios

In [42]:
test_data.head()

,sale_id,customer_id,category,amount,sale_date
0,1,106,Home,229,2024-01-01
1,2,103,Clothes,575,2024-01-02
2,3,107,Electronics,864,2024-01-03
3,4,104,Food,745,2024-01-04
4,5,106,Beauty,234,2024-01-05


1. Numerar las ventas de cada cliente.

📌 Enunciado:
Queremos asignar un número de fila a cada venta de un cliente en orden cronológico. Usa ROW_NUMBER() para numerar cada venta de un cliente, ordenándolas por fecha de venta.

In [44]:
query = """
SELECT 
    customer_id,
    sale_date,
    amount,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY sale_date)
FROM datos_transaccionales;
"""
ejercicio_1 = execute_query(query, credenciales)

In [51]:
(
    ejercicio_1
    .query('customer_id == 106')
    .sort_values('sale_date')
)

,customer_id,sale_date,amount,row_number
51,106,2024-01-01 00:00:00,229,1
52,106,2024-01-05 00:00:00,234,2
53,106,2024-01-08 00:00:00,647,3
54,106,2024-01-29 00:00:00,207,4
55,106,2024-02-04 00:00:00,618,5
56,106,2024-02-07 00:00:00,196,6
57,106,2024-02-18 00:00:00,972,7
58,106,2024-03-09 00:00:00,160,8
59,106,2024-03-21 00:00:00,11,9
60,106,2024-04-04 00:00:00,140,10


2. Calcular el ranking de ventas por categoría.

📌 Enunciado:
Determina el ranking de ventas dentro de cada categoría, ordenando por monto de venta en orden descendente. Usa RANK() para asignar el puesto de cada venta dentro de su categoría.

In [57]:
query = """
SELECT 
    category,
    sale_date,
    amount,
    RANK() OVER (PARTITION BY category ORDER BY amount DESC)
FROM datos_transaccionales;
"""
ejercicio_2 = execute_query(query, credenciales)

In [58]:
(
    ejercicio_1
    .query('category == "Home"')
    .sort_values('amount', ascending=False)
)

,category,sale_date,amount,rank
26,Home,2024-01-30 00:00:00,991,1
51,Home,2024-02-18 00:00:00,972,1
45,Home,2024-02-27 00:00:00,945,1
27,Home,2024-02-22 00:00:00,873,2
18,Home,2024-01-28 00:00:00,619,2
2,Home,2024-03-19 00:00:00,581,3
71,Home,2024-02-19 00:00:00,520,10
31,Home,2024-03-07 00:00:00,482,6
20,Home,2024-01-07 00:00:00,412,4
83,Home,2024-02-10 00:00:00,358,7


3.  Comparar el monto de cada venta con la anterior.

📌 Enunciado:
Para cada venta, queremos ver cuánto vendió un cliente en su venta anterior. Usa LAG() para obtener el monto de la venta anterior de cada cliente.

4. Calcular la suma acumulada de ventas por cliente

📌 Enunciado:
Queremos calcular la suma acumulada del monto de ventas por cliente en orden cronológico. Usa SUM() con OVER() para obtener el total acumulado de cada cliente.

5. Obtener el promedio móvil de ventas por categoría

📌 Enunciado:
Calcula el promedio móvil de las últimas 3 ventas dentro de cada categoría. Usa AVG() con ROWS BETWEEN 2 PRECEDING AND CURRENT ROW para calcularlo.

In [60]:
test_data.head()

,sale_id,customer_id,category,amount,sale_date
0,1,106,Home,229,2024-01-01
1,2,103,Clothes,575,2024-01-02
2,3,107,Electronics,864,2024-01-03
3,4,104,Food,745,2024-01-04
4,5,106,Beauty,234,2024-01-05


In [70]:
query = """
SELECT 
    customer_id,
    sale_date,
    amount,
    AVG(amount) OVER (PARTITION BY customer_id ORDER BY sale_date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)
FROM datos_transaccionales;
"""
ejemplo3 = execute_query(query, credenciales)

In [ ]:
(
    ejemplo3
    .query('customer_id == 100') 
    .sort_values('sale_date')   
)

,customer_id,sale_date,amount,avg
0,100,2024-01-22 00:00:00,410,410.0000000000000000
1,100,2024-01-26 00:00:00,846,628.0000000000000000
2,100,2024-02-21 00:00:00,157,471.0000000000000000
3,100,2024-03-19 00:00:00,581,528.0000000000000000
4,100,2024-03-24 00:00:00,263,333.6666666666666667
5,100,2024-03-28 00:00:00,169,337.6666666666666667
6,100,2024-04-01 00:00:00,668,366.6666666666666667


In [71]:
query = """
SELECT 
    customer_id,
    sale_date,
    amount,
    SUM(amount) OVER (PARTITION BY customer_id ORDER BY sale_date ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING)
FROM datos_transaccionales;
"""
ejemplo4 = execute_query(query, credenciales)

In [73]:
(
    ejemplo4
    .query('customer_id == 100')
    .sort_values('sale_date')
)

,customer_id,sale_date,amount,sum
0,100,2024-01-22 00:00:00,410,1256
1,100,2024-01-26 00:00:00,846,1413
2,100,2024-02-21 00:00:00,157,1584
3,100,2024-03-19 00:00:00,581,1001
4,100,2024-03-24 00:00:00,263,1013
5,100,2024-03-28 00:00:00,169,1100
6,100,2024-04-01 00:00:00,668,837


# Respuestas

1. 

```sql

SELECT 
    customer_id, 
    sale_id, 
    amount, 
    sale_date,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY sale_date) AS sale_number
FROM sales;

```

2. 

```sql

SELECT 
    category, 
    sale_id, 
    amount, 
    RANK() OVER (PARTITION BY category ORDER BY amount DESC) AS sale_rank
FROM sales;

```

3. 

```sql

SELECT 
    customer_id, 
    sale_id, 
    amount, 
    sale_date,
    LAG(amount) OVER (PARTITION BY customer_id ORDER BY sale_date) AS previous_sale
FROM sales;


```

4. 

```sql

SELECT 
    customer_id, 
    sale_id, 
    amount, 
    sale_date,
    SUM(amount) OVER (PARTITION BY customer_id ORDER BY sale_date) AS cumulative_sales
FROM sales;

```

5. 

```sql

SELECT 
    category, 
    sale_id, 
    amount, 
    sale_date,
    AVG(amount) OVER (PARTITION BY category ORDER BY sale_date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS moving_avg
FROM sales;


```